# Mount my drive:

In [1]:
#Import drive
from google.colab import drive
#Mount Google Drive
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir('drive/My Drive/Work/Thesis_Julien_Dejasmin/Work/code/Binary_activations_V2/MNIST_Binary_V2')
!ls

config.py   distributions  README.md	     trained_models
data	    experiments    requirements.txt  utils
DataLoader  __pycache__    results	     visualize


# Import:

In [14]:
!pip install pytorch-ignite

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
try:
  from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
  from ignite.metrics import Accuracy, Loss, ConfusionMatrix
  from ignite.handlers import ModelCheckpoint
  from utils.training import run, evaluate
except ImportError:
  raise RuntimeError("no module Ignite, to install Ignite: 'pip install pytorch-ignite'.")

from tqdm import tqdm

In [0]:
import torch
from torch import nn
from torch.optim import SGD
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.transforms import Compose, ToTensor, Normalize
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import numpy as np
from functools import partial

from utils.models import get_my_model_MNIST, fetch_last_checkpoint_model_filename
from DataLoader.dataLoaders import get_mnist_dataloaders
from utils.functions import Hardsigmoid

In [0]:
from visualize.viz import visTensor, get_activation, viz_activations, viz_filters
from visualize.viz import viz_heatmap, test_predict_few_examples, standardize_and_clip, format_for_plotting
from visualize.viz import apply_transforms, GradientAscent, get_filter_layer2, get_labels_histogram
from visualize.viz import get_region_layer1, get_region_layer2, get_regions_interest, get_all_regions_max
from visualize.viz import compare_two_histograms,viz_regions

# for regions extraction
import collections
from functools import partial
import cv2

# Load models and data:

In [11]:
batch_size_train = 10000
batch_size_test = 1000
# Dataset
train_loader, valid_loader, test_loader, classes = get_mnist_dataloaders(batch_size_train, batch_size_test)

Number of validation examples: 6000
Number of training examples: 6
Number of testing examples: 10


In [21]:
# load model pre trained no binary
binary = False
maxpooling = False

model_no_binary_stride, name_model = get_my_model_MNIST(binary, maxpooling=maxpooling)

path_model = 'trained_models/MNIST/No_binary_models/with_bias/stride'
if torch.cuda.is_available():
  model_no_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_no_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model)

Model Loaded MNIST_NonBinaryNet


In [19]:
# load model pre trained no binary
binary = False
maxpooling = True

model_no_binary_maxpooling, name_model = get_my_model_MNIST(binary, maxpooling=maxpooling)

path_model = 'trained_models/MNIST/No_binary_models/with_bias/maxpooling'
if torch.cuda.is_available():
  model_no_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_no_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model)

Model Loaded MNIST_NonBinaryNet_maxpooling


In [22]:
# load model pre trained binary
binary = True    
maxpooling = False
model_binary_stride, name_model = get_my_model_MNIST(binary, maxpooling=maxpooling)

path_model = 'trained_models/MNIST/Binary_models/with_bias/stride'
if torch.cuda.is_available():
  model_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_binary_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model)

Model Loaded MNIST_Stochastic_ST_first_conv_binary


In [23]:
# load model pre trained binary
binary = True    
maxpooling = True
model_binary_maxpooling, name_model = get_my_model_MNIST(binary, maxpooling=maxpooling)

path_model = 'trained_models/MNIST/Binary_models/with_bias/maxpooling'
if torch.cuda.is_available():
  model_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_binary_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))
print("Model Loaded", name_model)

Model Loaded MNIST_Stochastic_ST_first_conv_binary_maxpooling


In [24]:
# load model pre trained binary
binary = False
mixt = True    
maxpooling = False

model_mixt_stride, name_model = get_my_model_MNIST(binary, maxpooling=maxpooling, mixt=mixt)

path_model = 'trained_models/MNIST/Mixt_models/stride'
if torch.cuda.is_available():
  model_mixt_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_mixt_stride.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))

print("Model Loaded", name_model)

Model Loaded MNIST_Stochastic_ST_mixt


In [25]:
# load model pre trained binary
binary = False
mixt = True    
maxpooling = True

model_mixt_maxpooling, name_model = get_my_model_MNIST(binary, maxpooling=maxpooling, mixt=mixt)

path_model = 'trained_models/MNIST/Mixt_models/maxpooling'
if torch.cuda.is_available():
  model_mixt_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model)))
else:
  model_mixt_maxpooling.load_state_dict(torch.load(fetch_last_checkpoint_model_filename(path_model), map_location=torch.device('cpu')))

print("Model Loaded", name_model)

Model Loaded MNIST_Stochastic_ST_maxpooling_mixt


## Recap Scores:

In [26]:
evaluate(model_no_binary_stride, test_loader)
evaluate(model_no_binary_maxpooling, test_loader)
evaluate(model_binary_stride, test_loader)
evaluate(model_binary_maxpooling, test_loader)
evaluate(model_mixt_stride, test_loader)
evaluate(model_mixt_maxpooling, test_loader)

Test Results - Avg accuracy: 98.07 Avg loss: 0.06
Test Results - Avg accuracy: 98.59 Avg loss: 0.04
Test Results - Avg accuracy: 96.71 Avg loss: 0.11
Test Results - Avg accuracy: 97.57 Avg loss: 0.08
Test Results - Avg accuracy: 97.49 Avg loss: 0.08
Test Results - Avg accuracy: 98.49 Avg loss: 0.05


# Preleminary vews:

## Visualization few predictions:

In [0]:
print('No binary model with stride')
test_predict_few_examples(model_no_binary_stride, test_loader)
plt.show()
print('Binary model with stride')
test_predict_few_examples(model_binary_stride, test_loader)
plt.show()
print('Mixt model with stride')
test_predict_few_examples(model_mixt_stride, test_loader)
plt.show()
print('No binary model with maxpooling')
test_predict_few_examples(model_no_binary_maxpooling, test_loader)
plt.show()
print('Binary model with maxpooling')
test_predict_few_examples(model_binary_maxpooling, test_loader)
plt.show()
print('Mixt model with maxpooling')
test_predict_few_examples(model_mixt_maxpooling, test_loader)
plt.show()

## Visualization Activations values for a specific data:

In [0]:
index_data = 10

In [0]:
print('No binary model with stride')
viz_activations(model_no_binary_stride, test_loader, index_data)
print('Binary model with stride')
viz_activations(model_binary_stride, test_loader, index_data)
print('Mixt model with stride')
viz_activations(model_mixt_stride, test_loader, index_data)
print('No binary model with maxpooling')
viz_activations(model_no_binary_maxpooling, test_loader, index_data)
print('Binary model with maxpooling')
viz_activations(model_binary_maxpooling, test_loader, index_data)
print('Mixt model with maxpooling')
viz_activations(model_mixt_maxpooling, test_loader, index_data)

## Visualization heatmap for a specific data:

In [0]:
index_data = 10

In [0]:
print('No binary model with stride')
viz_heatmap(model_no_binary_stride, name_model, test_loader, index_data)

In [0]:
print('Binary model with stride')
viz_heatmap(model_binary_stride, name_model, test_loader, index_data)

In [0]:
print('Mixt model with stride')
viz_heatmap(model_mixt_stride, name_model, test_loader, index_data)

In [0]:
print('No binary model with maxpooling')
viz_heatmap(model_no_binary_maxpooling, name_model, test_loader, index_data)

In [0]:
print('Binary model with maxpooling')
viz_heatmap(model_binary_maxpooling, name_model, test_loader, index_data)

In [0]:
print('Mixt model with maxpooling')
viz_heatmap(model_mixt_maxpooling, name_model, test_loader, index_data)

## Visualization filters trained:

In [0]:
print('No binary model with stride')
viz_filters(model_no_binary_stride)

In [0]:
print('Binary model with stride')
viz_filters(model_binary_stride)

In [0]:
print('Mixt model with stride')
viz_filters(model_mixt_stride)

In [0]:
print('No binary model with maxpooling')
viz_filters(model_no_binary_maxpooling)

In [0]:
print('Binary model with maxpooling')
viz_filters(model_binary_maxpooling)

In [0]:
print('Mixt model with maxpooling')
viz_filters(model_mixt_maxpooling)

## Visualization generated images for activation maximization with gradient a ascent:

Activation maximization is one form of feature visualization that allows us to visualize what CNN filters are "looking for", by applying each filter to an input image and updating the input image so as to maximize the activation of the filter of interest (i.e. treating it as a gradient ascent task with filter activation values as the loss).

In [0]:
# parameters for MNIST data:

MNIST = True

nb_channels=1
img_size=28

conv1_filters = [0,1,2,3,4,5,6,7,8,9]
conv2_filters = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

mean_gradient_layer1 = False
mean_gradient_layer2 = False

ind_x_layer1 = 7
ind_y_layer1 = 7

ind_x_layer2 = 3
ind_y_layer2 = 3

lr = 0.001
num_iter = 100

### No binary model:

#### With stride:

In [0]:
g_ascent_no_binary_stride = GradientAscent(model_no_binary_stride, nb_channels=nb_channels, img_size=img_size)
g_ascent_no_binary_stride.use_gpu = True

In [0]:
conv1_no_binary_stride = model_no_binary_stride.layer1
conv2_no_binary_stride = model_no_binary_stride.layer2

In [0]:
g_ascent_no_binary_stride.visualize(conv1_no_binary_stride, MNIST, conv1_filters, mean_gradient_layer1,
                             ind_x_layer1, ind_y_layer1, lr=lr, num_iter=num_iter, title='No binary with stride model: conv layer 1')
g_ascent_no_binary_stride.visualize(conv2_no_binary_stride, MNIST, conv2_filters, mean_gradient_layer2,
                             ind_x_layer2, ind_y_layer2, lr=lr, num_iter=num_iter, title='No binary with stride model: conv layer 2')

#### With maxpooling:

In [0]:
g_ascent_no_binary_maxpooling = GradientAscent(model_no_binary_maxpooling, nb_channels=nb_channels, img_size=img_size)
g_ascent_no_binary_maxpooling.use_gpu = True

In [0]:
conv1_no_binary_maxpooling = model_no_binary_maxpooling.layer1
conv2_no_binary_maxpooling = model_no_binary_maxpooling.layer2

In [0]:
g_ascent_no_binary_maxpooling.visualize(conv1_no_binary_maxpooling, MNIST, conv1_filters, mean_gradient_layer1,
                             ind_x_layer1, ind_y_layer1, lr=lr, num_iter=num_iter, title='No binary with maxpooling model: conv layer 1')
g_ascent_no_binary_maxpooling.visualize(conv2_no_binary_maxpooling, MNIST, conv2_filters, mean_gradient_layer2,
                             ind_x_layer2, ind_y_layer2, lr=lr, num_iter=num_iter, title='No binary with maxpooling model: conv layer 2')

### Binary model:

#### With stride:

In [0]:
g_ascent_binary_stride = GradientAscent(model_binary_stride, nb_channels=nb_channels, img_size=img_size)
g_ascent_binary_stride.use_gpu = True

In [0]:
conv1_binary_stride = model_binary_stride.layer1
conv2_binary_stride = model_binary_stride.layer2

In [0]:
g_ascent_binary_stride.visualize(conv1_binary_stride, MNIST, conv1_filters, mean_gradient_layer1,
                             ind_x_layer1, ind_y_layer1, lr=lr, num_iter=num_iter, title='Binary with stride model: conv layer 1')
g_ascent_binary_stride.visualize(conv2_binary_stride, MNIST, conv2_filters, mean_gradient_layer2,
                             ind_x_layer2, ind_y_layer2, lr=lr, num_iter=num_iter, title='Binary with stride model: conv layer 2')

#### With Maxpooling:

In [0]:
g_ascent_binary_maxpooling = GradientAscent(model_binary_maxpooling, nb_channels=nb_channels, img_size=img_size)
g_ascent_binary_maxpooling.use_gpu = True

In [0]:
conv1_binary_maxpooling = model_binary_maxpooling.layer1
conv2_binary_maxpooling = model_binary_maxpooling.layer2

In [0]:
g_ascent_binary_maxpooling.visualize(conv1_binary_maxpooling, MNIST, conv1_filters, mean_gradient_layer1,
                             ind_x_layer1, ind_y_layer1, lr=lr, num_iter=num_iter, title='Binary with maxpooling model: conv layer 1')
g_ascent_binary_maxpooling.visualize(conv2_binary_maxpooling, MNIST, conv2_filters, mean_gradient_layer2,
                             ind_x_layer2, ind_y_layer2, lr=lr, num_iter=num_iter, title='Binary with maxpooling model: conv layer 2')

### Mixt model:

#### With stride:

In [0]:
g_ascent_mixt_stride = GradientAscent(model_mixt_stride, nb_channels=nb_channels, img_size=img_size)
g_ascent_mixt_stride.use_gpu = True

In [0]:
conv1_mixt_stride = model_mixt_stride.layer1
conv2_mixt_stride = model_mixt_stride.layer2

In [0]:
g_ascent_mixt_stride.visualize(conv1_mixt_stride, MNIST, conv1_filters, mean_gradient_layer1,
                             ind_x_layer1, ind_y_layer1, lr=lr, num_iter=num_iter, title='Mixt with stride model: conv layer 1')
g_ascent_mixt_stride.visualize(conv2_mixt_stride, MNIST, conv2_filters, mean_gradient_layer2,
                             ind_x_layer2, ind_y_layer2, lr=lr, num_iter=num_iter, title='Mixt with stride model: conv layer 2')

#### With Maxpooling:

In [0]:
g_ascent_mixt_maxpooling = GradientAscent(model_mixt_maxpooling, nb_channels=nb_channels, img_size=img_size)
g_ascent_mixt_maxpooling.use_gpu = True

In [0]:
conv1_mixt_maxpooling = model_mixt_maxpooling.layer1
conv2_mixt_maxpooling = model_mixt_maxpooling.layer2

In [0]:
g_ascent_mixt_maxpooling.visualize(conv1_mixt_maxpooling, MNIST, conv1_filters, mean_gradient_layer1,
                             ind_x_layer1, ind_y_layer1, lr=lr, num_iter=num_iter, title='Mixt with maxpooling model: conv layer 1')
g_ascent_mixt_maxpooling.visualize(conv2_mixt_maxpooling, MNIST, conv2_filters, mean_gradient_layer2,
                             ind_x_layer2, ind_y_layer2, lr=lr, num_iter=num_iter, title='Mixt with maxpooling model: conv layer 2')

# Visualize Regions that maximized filters:

## Load regions, activations and regions means:

In [0]:
# Labels:
labels = np.load('results/MNIST_results/MNIST_regions/labels.npy', allow_pickle=True)

# No binary model with stride:
region_layer1_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/No_binary_model/stride/regions_max_layer1.npy', allow_pickle=True)
region_layer2_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/No_binary_model/stride/regions_max_layer2.npy', allow_pickle=True)
activation_layer1_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/No_binary_model/stride/activations_max_layer1.npy', allow_pickle=True)
activation_layer2_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/No_binary_model/stride/activations_max_layer2.npy', allow_pickle=True)
activation_layer1_no_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/No_binary_model/stride/activations_max_layer1_normalized.npy', allow_pickle=True)
activation_layer2_no_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/No_binary_model/stride/activations_max_layer2_normalized.npy', allow_pickle=True)

# No binary model with maxpooling:
region_layer1_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/No_binary_model/maxpooling/regions_max_layer2.npy', allow_pickle=True)
region_layer2_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/No_binary_model/maxpooling/regions_max_layer2.npy', allow_pickle=True)
activation_layer1_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/No_binary_model/maxpooling/activations_max_layer1.npy', allow_pickle=True)
activation_layer2_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/No_binary_model/maxpooling/activations_max_layer2.npy', allow_pickle=True)
activation_layer1_no_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/No_binary_model/maxpooling/activations_max_layer1_normalized.npy', allow_pickle=True)
activation_layer2_no_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/No_binary_model/maxpooling/activations_max_layer2_normalized.npy', allow_pickle=True)

# Binary model with stride:
region_layer1_binary_stride = np.load('results/MNIST_results/MNIST_regions/Binary_model/stride/regions_max_layer1.npy', allow_pickle=True)
region_layer2_binary_stride = np.load('results/MNIST_results/MNIST_regions/Binary_model/stride/regions_max_layer2.npy', allow_pickle=True)
activation_layer1_binary_stride = np.load('results/MNIST_results/MNIST_regions/Binary_model/stride/activations_max_layer1.npy', allow_pickle=True)
activation_layer2_binary_stride = np.load('results/MNIST_results/MNIST_regions/Binary_model/stride/activations_max_layer2.npy', allow_pickle=True)
activation_layer1_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/Binary_model/stride/activations_max_layer1_normalized.npy', allow_pickle=True)
activation_layer2_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/Binary_model/stride/activations_max_layer2_normalized.npy', allow_pickle=True)

# Binary model with maxpooling:
region_layer1_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Binary_model/maxpooling/regions_max_layer1.npy', allow_pickle=True)
region_layer2_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Binary_model/maxpooling/regions_max_layer2.npy', allow_pickle=True)
activation_layer1_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Binary_model/maxpooling/activations_max_layer1.npy', allow_pickle=True)
activation_layer2_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Binary_model/maxpooling/activations_max_layer2.npy', allow_pickle=True)
activation_layer1_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/Binary_model/maxpooling/activations_max_layer1_normalized.npy', allow_pickle=True)
activation_layer2_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/Binary_model/maxpooling/activations_max_layer2_normalized.npy', allow_pickle=True)

# Mixt model no binary with stride:
region_layer1_mixt_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/regions_max_layer1_no_binary.npy', allow_pickle=True)
region_layer2_mixt_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/regions_max_layer2_no_binary.npy', allow_pickle=True)
activation_layer1_mixt_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer1_no_binary.npy', allow_pickle=True)
activation_layer2_mixt_no_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer2_no_binary.npy', allow_pickle=True)
activation_layer1_mixt_no_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer1_no_binary_normalized.npy', allow_pickle=True)
activation_layer2_mixt_no_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer2_no_binary_normalized.npy', allow_pickle=True)

# Mixt model binary with stride:
region_layer1_mixt_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/regions_max_layer1_binary.npy', allow_pickle=True)
region_layer2_mixt_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/regions_max_layer2_binary.npy', allow_pickle=True)
activation_layer1_mixt_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer1_binary.npy', allow_pickle=True)
activation_layer2_mixt_binary_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer2_binary.npy', allow_pickle=True)
activation_layer1_mixt_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer1_binary_normalized.npy', allow_pickle=True)
activation_layer2_mixt_binary_normalized_stride = np.load('results/MNIST_results/MNIST_regions/Mixt_model/stride/activations_max_layer2_binary_normalized.npy', allow_pickle=True)

# Mixt model no binary with maxpooling:
region_layer1_mixt_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/regions_max_layer1_no_binary.npy', allow_pickle=True)
region_layer2_mixt_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/regions_max_layer2_no_binary.npy', allow_pickle=True)
activation_layer1_mixt_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer1_no_binary.npy', allow_pickle=True)
activation_layer2_mixt_no_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer2_no_binary.npy', allow_pickle=True)
activation_layer1_mixt_no_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer1_no_binary_normalized.npy', allow_pickle=True)
activation_layer2_mixt_no_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer2_no_binary_normalized.npy', allow_pickle=True)

# Mixt model binary with maxpooling:
region_layer1_mixt_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/regions_max_layer1_binary.npy', allow_pickle=True)
region_layer2_mixt_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/regions_max_layer2_binary.npy', allow_pickle=True)
activation_layer1_mixt_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer1_binary.npy', allow_pickle=True)
activation_layer2_mixt_binary_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer2_binary.npy', allow_pickle=True)
activation_layer1_mixt_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer1_binary_normalized.npy', allow_pickle=True)
activation_layer2_mixt_binary_normalized_maxpooling = np.load('results/MNIST_results/MNIST_regions/Mixt_model/maxpooling/activations_max_layer2_binary_normalized.npy', allow_pickle=True)

## Visualize regions details:

In [0]:
# parameters
list_filter_interest_layer1 = [0,1,2,3,4,5,6,7,8,9]
list_filter_interest_layer2 = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

best = True
worst = False
viz_mean_img = True
viz_grid = True
plot_histogram = True

percentage = 10
nrow=14

In [0]:
# regions and activation of interest
list_filter = list_filter_interest_layer1
regions = None
activations = None
activations_normalized = None

In [0]:
selected_regions, activation_values, activation_values_normalized = get_regions_interest(regions, labels, activations, activations_normalized, details=True, best, worst, viz_mean_img, viz_grid, percentage, list_filter, nrow=nrow, plot_histogram=plot_histogram)

## Histograms:

In [0]:
plt.hist(labels, bins=20)

In [0]:
# get histograms:
activation = None
activations_normalized = None
list_filter = range(10)  # or 20
best = True
percentage = 10
return_values = True
plot = False

In [0]:
_, labels_selected, labels_selected_normalized, labels_selected_all, labels_selected_all_normalized = get_labels_histogram(labels, activation, activations_normalized, list_filter=list_filter, best=best, percentage=percentage, plot=plot, return_values=return_values)

### Compare two histograms:

In [0]:
x = None
y = None
label_x = 'title x'
label_y ='title 2'

bins = np.linspace(0, 10, 20)

plt.figure(num=None, figsize=(10, 8), dpi=80, facecolor='w', edgecolor='k')
plt.hist([x, y], bins, alpha=0.75, label=[label_x, label_y])
plt.legend(loc='upper right')
plt.title('Title')
plt.show()

## Visualize resume regions:

In [0]:
#TODO: function to print means regions with parameters 'percentage' for all networks with clearly print. Visualize also filters learned.
# use parameters 'print details' or 'print resume'.

In [0]:
regions = region_layer1_no_binary_stride
activation = activation_layer1_no_binary_stride
activations_normalized = activation_layer1_no_binary_normalized_stride

resume_mean_regions(regions, labels, activation, activations_normalized)

In [0]:
regions = region_layer2_no_binary_stride
activation = activation_layer2_no_binary_stride
activations_normalized = activation_layer2_no_binary_normalized_stride

resume_mean_regions(regions, labels, activation, activations_normalized)